# Imports + glove download

In [59]:
%load_ext autoreload
%autoreload 2
train_filename = r'..\data\train.tagged'
dev_filename = r'..\data\dev.tagged'
import numpy as np
from hw1_help_funcs import read_file, data_to_vectors, add_context, sep_X_y, set_seed, data_to_vectors_extra_features, agg_context_mean
from gensim import downloader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from gensim.models import Word2Vec
import pickle
from sklearn.metrics import f1_score


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
GLOVE_PATH = 'glove-twitter-200'
glove = downloader.load(GLOVE_PATH)

In [2]:
# to be removed
with open('glove.pickle', 'rb') as handle: 
    glove = pickle.load(handle)

# Loading and vectorizing the train and dev data, then creating X and y for the models

In [ ]:
data = read_file(filename)
data[:10]

In [5]:
vec_data = data_to_vectors(data, glove)
vec_data[:10]

[[[array([-0.76362962, -0.09652831,  0.24288038,  0.82802042, -0.51917588,
          -0.29963783, -0.4824618 ,  0.8086042 ,  0.74575822,  0.35823595,
          -0.76197669, -0.3632796 ,  0.63579972, -0.3954818 ,  0.32237537,
           0.60976341, -0.69141938, -0.16793046,  0.64874354,  0.13477284,
           0.95353931,  0.29524489,  0.63457181,  0.57929568, -0.53320487,
           0.33509422,  0.1923611 ,  0.53788352,  0.24061702, -0.44425806,
          -0.47477013,  0.05402432, -0.25234933,  0.14056346,  0.0237509 ,
           0.50818291, -0.59879472, -0.98458888,  0.07465946, -0.74942623,
          -0.06649493,  0.56484295,  0.95867872,  0.8102586 ,  0.77951239,
           0.94394352, -0.29717563,  0.576841  ,  0.76040694,  0.69695323,
          -0.17901636,  0.85586991, -0.50689875, -0.49618194,  0.91462729,
           0.63711727,  0.30865206,  0.54268287, -0.17015458,  0.26890278,
          -0.48368709,  0.67699487,  0.03703159, -0.55939812, -0.21322557,
          -0.96340001,  0

In [31]:
dev_vec_data = data_to_vectors(read_file(r'..\data\dev.tagged'), glove)

In [33]:
X_train, y_train = sep_X_y(vec_data)
X_dev, y_dev = sep_X_y(dev_vec_data)

# SVM simple model

In [615]:
from sklearn import svm
svm_model = svm.SVC()
svm_model.fit(X_train, y_train)
dev_svm_predictions = svm_model.predict(X_dev)
f1_score(y_dev, dev_svm_predictions)

0.5691056910569107

# NN simple model

In [609]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_dim = 10
        self.layer_1 = torch.nn.Linear(200, self.hidden_dim)
        self.layer_2 = torch.nn.Linear(self.hidden_dim, 1)
        self.activation = F.relu

    def forward(self, x):
    x = self.layer_1(x)        # x.size() -> [batch_size, self.hidden_dim]
    x = self.activation(x)     # x.size() -> [batch_size, self.hidden_dim]
    x = self.layer_2(x)        # x.size() -> [batch_size, 1]
    x = torch.sigmoid(x)
    return x  

net = Network()
print(net)

Network(
  (layer_1): Linear(in_features=200, out_features=10, bias=True)
  (layer_2): Linear(in_features=10, out_features=1, bias=True)
)


In [610]:
# Define Optimizer and Loss Function
optimizer = torch.optim.SGD(net.parameters(), lr=0.2)
loss_func = torch.nn.MSELoss()

In [611]:
set_seed()
batch_size = 20

# create Tensor datasets
train_data = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
dev_data = TensorDataset(torch.FloatTensor(X_dev), torch.FloatTensor(y_dev))


# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(dev_data, shuffle=False, batch_size=batch_size)


In [612]:
# Define training params
epochs = 10

counter = 0
print_every = 1000
clip = 1000 # gradient clipping

net.train()
# train for some number of epochs

Network(
  (layer_1): Linear(in_features=200, out_features=10, bias=True)
  (layer_2): Linear(in_features=10, out_features=1, bias=True)
)

In [613]:
for e in range(epochs):
    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        # x.size() -> [batch_size]
        batch_size = inputs.size(0)
        # IMPORTANT - change the dimensions of x before it enters the NN, batch size must always be first
        x = inputs.unsqueeze(0)         # x.size() -> [1, batch_size]
        x = x.view(batch_size, -1)      # x.size() -> [batch_size, 1]
        predictions = net(x)

        # calculate the loss and perform backprop
        loss = loss_func(predictions.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_losses = []
            net.eval()
            print_flag = True
            for inputs, labels in valid_loader:
                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                 

                # get the output from the model
                # x.size() -> [batch_size]
                batch_size = inputs.size(0)
                # IMPORTANT - change the dimensions of x before it enters the NN, batch size must always be first
                x = inputs.unsqueeze(0)    # x.size() -> [1, batch_size]
                x = x.view(batch_size, -1) # x.size() -> [batch_size, 1]
                val_predictions = net(x)
                val_loss = loss_func(val_predictions.squeeze(), labels.float())
                val_losses.append(val_loss.item())
                

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

C:\Users\Ziv Keidar\anaconda3\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 1/10... Step: 1000... Loss: 0.015103... Val Loss: 0.049369
Epoch: 1/10... Step: 2000... Loss: 0.001965... Val Loss: 0.047638
Epoch: 2/10... Step: 3000... Loss: 0.012960... Val Loss: 0.046757
Epoch: 2/10... Step: 4000... Loss: 0.022363... Val Loss: 0.047147
Epoch: 3/10... Step: 5000... Loss: 0.005834... Val Loss: 0.047217
Epoch: 3/10... Step: 6000... Loss: 0.010354... Val Loss: 0.046858
Epoch: 4/10... Step: 7000... Loss: 0.049329... Val Loss: 0.045913
Epoch: 4/10... Step: 8000... Loss: 0.034860... Val Loss: 0.045504
Epoch: 4/10... Step: 9000... Loss: 0.051018... Val Loss: 0.045670
Epoch: 5/10... Step: 10000... Loss: 0.052199... Val Loss: 0.045002
Epoch: 5/10... Step: 11000... Loss: 0.000083... Val Loss: 0.045554
Epoch: 6/10... Step: 12000... Loss: 0.064758... Val Loss: 0.045774
Epoch: 6/10... Step: 13000... Loss: 0.000104... Val Loss: 0.045599
Epoch: 7/10... Step: 14000... Loss: 0.000819... Val Loss: 0.044047
Epoch: 7/10... Step: 15000... Loss: 0.050361... Val Loss: 0.045137
Epoc

In [614]:
x = torch.FloatTensor(X_dev).unsqueeze(0)
x = x.view(len(X_dev), -1)
nn_predictions = torch.round(net(x)).detach().numpy()
f1_score(y_dev, nn_predictions)

0.5260869565217391

# Code to add the context to the vectorized data and make X from it - train and dev (600 length voctors)

In [ ]:
vec_data[0][0][0].shape

In [84]:
# vec_data = vec_Train
# vec_length = vec_data[0][0][0].shape[0] 
# length = 1
# context_size = 3
# set_seed()
# sen_seperator = np.random.uniform(-1, 1, vec_length)
# for sen in vec_data:
#     for i, word in enumerate(sen):
#         length += context_size
#         if i == len(sen) - 1:
#             length += context_size
# one_vec_data = np.zeros(length*vec_length)
# jump = 0
# index = 0
# non_data_idx = []
# np.put(one_vec_data, np.arange(len(one_vec_data)-vec_length, len(one_vec_data)), sen_seperator)
# for i, sen in enumerate(vec_data):
#     for j, word in enumerate(sen):
#         if j == 0:
#             for i in range(context_size):
#                 non_data_idx.append(index+jump)
#                 np.put(one_vec_data, np.arange(vec_length*(index+jump), 
#                                                vec_length*(index+jump)+vec_length), sen_seperator)
#                 jump += 1

#         np.put(one_vec_data, np.arange(vec_length*(index+jump), 
#                                        vec_length*(index+jump)+vec_length), word[0]) 
#         index += 1
# one_vec_data.shape

(30050950,)

In [72]:
a[0][0]

array([1, 2, 3])

In [51]:
a = [[[np.array([1, 2]), 1], [np.array([3, 4]), 0], [np.array([5, 6]), 1]], 
      [[np.array([7, 8]), 1], [np.array([9, 0]), 0]]]
set_seed()
vec_length = a[0][0][0].shape[0]
sen_seperator = np.zeros(vec_length)
X_train = add_context(a, context_size=4, sen_separator=sen_seperator)
# X_dev = add_context(a, context_size=2, sen_separator=sen_seperator)


In [53]:
set_seed()
vec_length = vec_Train[0][0][0].shape[0]
sen_seperator = np.random.uniform(-1, 1, vec_length)
X_train = add_context(vec_Train, context_size=3, sen_separator=sen_seperator)
X_dev = add_context(vec_Dev, context_size=3, sen_separator=sen_seperator)


In [66]:
X_train_agg = agg_context_mean(X_train, context_size, vec_length)
X_dev_agg = agg_context_mean(X_dev, context_size, vec_length)

# NN for the 600 sized vectors- combined with weights for the classes

In [139]:
class Network2(nn.Module):
    def __init__(self, input_dim=600):
        super().__init__()
        self.hidden_dim1 = 100
        self.hidden_dim2 = 10
        self.layer_1 = torch.nn.Linear(input_dim, self.hidden_dim1)
        self.layer_2 = torch.nn.Linear(self.hidden_dim1, self.hidden_dim2)
        self.layer_3 = torch.nn.Linear(self.hidden_dim2, 1)
        self.activation = F.relu

    def forward(self, x):
        x = self.layer_1(x)        # x.size() -> [batch_size, self.hidden_dim]
        x = self.activation(x)     # x.size() -> [batch_size, self.hidden_dim]
        x = self.layer_2(x)        # x.size() -> [batch_size, 1]
        x = self.activation(x)
        x = self.layer_3(x)
        x = torch.sigmoid(x)
        return x  

net2 = Network2(input_dim=615)
print(net2)

Network2(
  (layer_1): Linear(in_features=615, out_features=100, bias=True)
  (layer_2): Linear(in_features=100, out_features=10, bias=True)
  (layer_3): Linear(in_features=10, out_features=1, bias=True)
)


In [140]:
# Define Optimizer and Loss Function
optimizer = torch.optim.SGD(net2.parameters(), lr=0.02)
loss_func = torch.nn.BCELoss(reduction='none')

In [141]:
set_seed()
batch_size = 16

# create Tensor datasets
new_train_data = TensorDataset(torch.FloatTensor(X_train_agg), torch.FloatTensor(y_train))
new_dev_data = TensorDataset(torch.FloatTensor(X_dev_agg), torch.FloatTensor(y_dev))


# make sure the SHUFFLE your training data
new_train_loader = DataLoader(new_train_data, shuffle=True, batch_size=batch_size)
new_valid_loader = DataLoader(new_dev_data, shuffle=False, batch_size=batch_size)

In [142]:
# Define training params
epochs = 10

counter = 0
print_every = 1000
clip = 1000 # gradient clipping

net2.train()
# train for some number of epochs


Network2(
  (layer_1): Linear(in_features=615, out_features=100, bias=True)
  (layer_2): Linear(in_features=100, out_features=10, bias=True)
  (layer_3): Linear(in_features=10, out_features=1, bias=True)
)

In [143]:
for e in range(epochs):
    # batch loop
    for inputs, labels in new_train_loader:
        counter += 1

        # zero accumulated gradients
        net2.zero_grad()

        # get the output from the model
        # x.size() -> [batch_size]
        batch_size = inputs.size(0)
        # IMPORTANT - change the dimensions of x before it enters the NN, batch size must always be first
        x = inputs.unsqueeze(0)         # x.size() -> [1, batch_size]
        x = x.view(batch_size, -1)      # x.size() -> [batch_size, 1]
        predictions = net2(x)
        # calculate the loss and perform backprop
        weights = torch.from_numpy(np.array([1 if i==1 else 0.7 for i in labels]))
        temp_loss = loss_func(predictions.squeeze(), labels.float().squeeze())
        loss = torch.mean(weights*temp_loss)
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net2.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_losses = []
            net2.eval()
            print_flag = True
            for inputs, labels in new_valid_loader:
                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                 

                # get the output from the model
                # x.size() -> [batch_size]
                batch_size = inputs.size(0)
                # IMPORTANT - change the dimensions of x before it enters the NN, batch size must always be first
                x = inputs.unsqueeze(0)    # x.size() -> [1, batch_size]
                x = x.view(batch_size, -1) # x.size() -> [batch_size, 1]
                val_predictions = net2(x)
                val_weights = torch.from_numpy(np.array([1 if i==1 else 0.7 for i in labels]))
                temp_val_loss = loss_func(val_predictions.squeeze(), labels.float().squeeze())
                val_loss = torch.mean(val_weights*temp_val_loss)
                val_losses.append(val_loss.item())
                

            net2.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/10... Step: 1000... Loss: 0.107893... Val Loss: 0.179148
Epoch: 1/10... Step: 2000... Loss: 0.284717... Val Loss: 0.154158
Epoch: 2/10... Step: 3000... Loss: 0.063359... Val Loss: 0.146691
Epoch: 2/10... Step: 4000... Loss: 0.009410... Val Loss: 0.138925
Epoch: 2/10... Step: 5000... Loss: 0.105034... Val Loss: 0.139268
Epoch: 3/10... Step: 6000... Loss: 0.005242... Val Loss: 0.133677
Epoch: 3/10... Step: 7000... Loss: 0.305385... Val Loss: 0.138157
Epoch: 3/10... Step: 8000... Loss: 0.033114... Val Loss: 0.128415
Epoch: 4/10... Step: 9000... Loss: 0.024751... Val Loss: 0.141204
Epoch: 4/10... Step: 10000... Loss: 0.038289... Val Loss: 0.135284
Epoch: 4/10... Step: 11000... Loss: 0.034086... Val Loss: 0.121361
Epoch: 5/10... Step: 12000... Loss: 0.088987... Val Loss: 0.122633
Epoch: 5/10... Step: 13000... Loss: 0.039574... Val Loss: 0.120206
Epoch: 5/10... Step: 14000... Loss: 0.022974... Val Loss: 0.119814
Epoch: 6/10... Step: 15000... Loss: 0.292626... Val Loss: 0.133042
Epoc

In [144]:
x = torch.FloatTensor(X_dev_agg).unsqueeze(0)
x = x.view(len(X_dev_agg), -1)
nn_predictions = torch.round(net2(x)).detach().numpy()
f1_score(y_dev, nn_predictions)

0.6462668298653611

# NN simple model with weights

In [21]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_dim = 20
        self.layer_1 = torch.nn.Linear(205, self.hidden_dim)
        self.layer_2 = torch.nn.Linear(self.hidden_dim, 1)
        self.activation = F.relu

    def forward(self, x):
        x = self.layer_1(x)        # x.size() -> [batch_size, self.hidden_dim]
        x = self.activation(x)     # x.size() -> [batch_size, self.hidden_dim]
        x = self.layer_2(x)        # x.size() -> [batch_size, 1]
        x = torch.sigmoid(x)
        return x  

net = Network()
print(net)

Network(
  (layer_1): Linear(in_features=205, out_features=20, bias=True)
  (layer_2): Linear(in_features=20, out_features=1, bias=True)
)


In [22]:
# Define Optimizer and Loss Function
optimizer = torch.optim.SGD(net.parameters(), lr=0.02)
loss_func = torch.nn.BCELoss(reduction='none')

In [23]:
set_seed()
batch_size = 20

# create Tensor datasets
train_data = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
dev_data = TensorDataset(torch.FloatTensor(X_dev), torch.FloatTensor(y_dev))


# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(dev_data, shuffle=False, batch_size=batch_size)


In [24]:
# Define training params
epochs = 10

counter = 0
print_every = 1000
clip = 1000 # gradient clipping

net.train()
# train for some number of epochs

Network(
  (layer_1): Linear(in_features=205, out_features=20, bias=True)
  (layer_2): Linear(in_features=20, out_features=1, bias=True)
)

In [25]:
for e in range(epochs):
    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        # x.size() -> [batch_size]
        batch_size = inputs.size(0)
        # IMPORTANT - change the dimensions of x before it enters the NN, batch size must always be first
        x = inputs.unsqueeze(0)         # x.size() -> [1, batch_size]
        x = x.view(batch_size, -1)      # x.size() -> [batch_size, 1]
        predictions = net(x)

        # calculate the loss and perform backprop
        
        weights = torch.from_numpy(np.array([1 if i==1 else 0.8 for i in labels]))
        temp_loss = loss_func(predictions.squeeze(), labels.float().squeeze())
        loss = torch.mean(weights*temp_loss)
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_losses = []
            net.eval()
            print_flag = True
            for inputs, labels in valid_loader:
                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                 

                # get the output from the model
                # x.size() -> [batch_size]
                batch_size = inputs.size(0)
                # IMPORTANT - change the dimensions of x before it enters the NN, batch size must always be first
                x = inputs.unsqueeze(0)    # x.size() -> [1, batch_size]
                x = x.view(batch_size, -1) # x.size() -> [batch_size, 1]
                val_predictions = net(x)
                val_weights = torch.from_numpy(np.array([1 if i==1 else 0.8 for i in labels]))
                temp_val_loss = loss_func(val_predictions.squeeze(), labels.float().squeeze())
                val_loss = torch.mean(val_weights*temp_val_loss)
                val_losses.append(val_loss.item())
                

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/10... Step: 1000... Loss: 0.072174... Val Loss: 0.192125
Epoch: 1/10... Step: 2000... Loss: 0.040478... Val Loss: 0.169543
Epoch: 2/10... Step: 3000... Loss: 0.077328... Val Loss: 0.157295
Epoch: 2/10... Step: 4000... Loss: 0.060171... Val Loss: 0.152070
Epoch: 3/10... Step: 5000... Loss: 0.044884... Val Loss: 0.147281
Epoch: 3/10... Step: 6000... Loss: 0.036669... Val Loss: 0.145570
Epoch: 4/10... Step: 7000... Loss: 0.108550... Val Loss: 0.144550
Epoch: 4/10... Step: 8000... Loss: 0.071219... Val Loss: 0.144746
Epoch: 4/10... Step: 9000... Loss: 0.223922... Val Loss: 0.142106
Epoch: 5/10... Step: 10000... Loss: 0.369414... Val Loss: 0.144870
Epoch: 5/10... Step: 11000... Loss: 0.011470... Val Loss: 0.140260
Epoch: 6/10... Step: 12000... Loss: 0.099329... Val Loss: 0.141403
Epoch: 6/10... Step: 13000... Loss: 0.015127... Val Loss: 0.139386
Epoch: 7/10... Step: 14000... Loss: 0.076969... Val Loss: 0.138487
Epoch: 7/10... Step: 15000... Loss: 0.169601... Val Loss: 0.137661
Epoc

In [26]:
x = torch.FloatTensor(X_dev).unsqueeze(0)
x = x.view(len(X_dev), -1)
nn_predictions = torch.round(net(x)).detach().numpy()
f1_score(y_dev, nn_predictions)

0.6227141482194417

# Adding features and considering word2vec training on our train and dev

In [3]:
Train_data1 = read_file(train_filename)
Dev_data1 = read_file(dev_filename)
Train_data2 = read_file(train_filename)
Dev_data2 = read_file(dev_filename)

In [4]:
vec_Train = data_to_vectors_extra_features(Train_data1, Dev_data1, glove)
vec_Dev = data_to_vectors_extra_features(Dev_data2, Train_data2 ,glove)

In [10]:
X_train, y_train = sep_X_y(vec_Train)
X_dev, y_dev = sep_X_y(vec_Dev)

In [840]:
from sklearn import svm
svm_model = svm.SVC()
svm_model.fit(X_train, y_train)
dev_svm_predictions = svm_model.predict(X_dev)
f1_score(y_dev, dev_svm_predictions)

0.6201634877384197

In [841]:
glove[',']

array([ 0.3927   , -0.084181 , -0.6075   ,  0.3231   , -0.35919  ,
        0.62664  ,  0.29758  , -0.21039  , -0.23201  ,  0.11897  ,
        0.41827  , -0.21206  , -0.036036 ,  0.39526  ,  0.53432  ,
       -0.33607  , -0.23529  ,  0.027524 , -0.21025  , -0.49712  ,
        0.12486  , -0.14865  , -0.45697  ,  0.35445  , -0.31083  ,
       -1.8664   , -0.32149  , -0.41014  ,  0.35596  ,  0.25467  ,
       -0.35302  ,  0.059504 ,  0.1571   ,  0.61731  , -0.13229  ,
        0.18362  ,  0.1329   , -0.13754  , -0.068064 , -0.29138  ,
       -0.53293  , -0.048187 , -0.1334   ,  0.11968  , -0.045649 ,
       -0.42278  ,  0.69403  ,  0.30971  , -0.66289  ,  0.017076 ,
       -0.45864  , -0.23924  ,  0.24663  ,  0.13362  , -0.34587  ,
        0.031223 , -0.43579  ,  0.16547  , -0.25115  ,  0.48124  ,
       -0.21432  ,  0.22534  ,  0.20446  , -0.45843  ,  0.60826  ,
        0.20866  , -0.46438  , -0.34758  ,  0.088277 , -0.30944  ,
        0.41937  ,  0.12139  , -0.46349  ,  0.08518  , -0.0707

In [850]:
counter = 0
for sen in Dev_data1:
    for word in sen:
        if '@' in word[0]:
            counter += 1
            print(word[0], word[1])
print(counter)

@camerondallas 0
@Phil_Heim 0
@shashiranjanttv 0
@shashiranjanttv 0
@_SoleSurvivors 0
@jonnyharvey93 0
@LeBronJames 0
@jcshrader 0
@ilovebobong 0
@NopeFinger 0
@mdkso 0
@LUMserve 0
@ 0
@JUCOFFrenzy 0
@RealSteel223526 0
@njcaa 0
@null 0
@ramonmaster 0
@kjwgaminghd 0
@Jus2xtreme 0
@IAmJ3richoholic 0
@null 0
@macbby11 0
@ 0
@senegaloruskovv 0
@paulcoro 0
@jeanyousux 0
@tos 0
@Kukita_dance 0
@jarodzsz 0
@LusyLuLu 0
@Roshen 0
@wordpressdotcom 0
@_icze4r 0
@darcbowdler_ 0
@LF1N 0
@NajibRazak 0
@Sholea_Boland 0
@cyclingmoletips 0
@sharethis 0
@thefemaleboook 0
@AUBsesss 0
@nadinellanes 0
@_pat3 0
@Leeharmon 0
@speoples_ 0
@Meechie_Hoe215 0
@NastyNa215 0
@LucieMerl 0
@FitchRatings 0
@ 0
@happydaysjack99 0
@andywills1019 0
@DepressedDarth 0
@FinestSquad 0
@SarahCon14 0
@GOT7_JYP 0
@stinsonsays 0
@Harry_Styles 0
@null 0
@wisatatiketcom 0
@Butlins 0
@SportsCenter 0
@Icey_Roy 0
@NWAllenSchoolFW 0
@kelly_gingrich 0
@Wolven 0
@VanessaY 0
@miniver 0
@mathpunk 0
@NeilHarbisson 0
@ALADIN135 0
@lululemo